<a href="https://colab.research.google.com/github/21130503/data_mining/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [190]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import plotly.express as px

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score, balanced_accuracy_score

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model

In [191]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [192]:
%ls

breast-cancer.csv  Untitled0.ipynb


In [193]:
%cd '/content/drive/MyDrive/DM/baocao'

/content/drive/MyDrive/DM/baocao


In [194]:
df = pd.read_csv('breast-cancer.csv')

In [195]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [196]:
df.describe().T.style.background_gradient(cmap = sns.color_palette("ch:s=-.2,r=.6", as_cmap=True))

,count,mean,std,min,25%,50%,75%,max
id,569.000000,30371831.432337,125020585.612224,8670.000000,869218.000000,906024.000000,8813129.000000,911320502.000000
radius_mean,569.000000,14.127292,3.524049,6.981000,11.700000,13.370000,15.780000,28.110000
texture_mean,569.000000,19.289649,4.301036,9.710000,16.170000,18.840000,21.800000,39.280000
perimeter_mean,569.000000,91.969033,24.298981,43.790000,75.170000,86.240000,104.100000,188.500000
area_mean,569.000000,654.889104,351.914129,143.500000,420.300000,551.100000,782.700000,2501.000000
smoothness_mean,569.000000,0.096360,0.014064,0.052630,0.086370,0.095870,0.105300,0.163400
compactness_mean,569.000000,0.104341,0.052813,0.019380,0.064920,0.092630,0.130400,0.345400
concavity_mean,569.000000,0.088799,0.079720,0.000000,0.029560,0.061540,0.130700,0.426800
concave points_mean,569.000000,0.048919,0.038803,0.000000,0.020310,0.033500,0.074000,0.201200
symmetry_mean,569.000000,0.181162,0.027414,0.106000,0.161900,0.179200,0.195700,0.304000


In [197]:
px.histogram(data_frame=df, x='diagnosis', color='diagnosis',color_discrete_sequence=['#A865C9','#f6abb6'])

In [198]:
px.histogram(data_frame=df,x=df.area_mean,color='diagnosis',color_discrete_sequence=['#A865C9','#f6abb6'])


In [199]:
px.histogram(data_frame=df,x=df.texture_mean,color='diagnosis',color_discrete_sequence=['#A865C9','#f6abb6'])


In [200]:
px.histogram(data_frame=df,x=df.radius_mean,color='diagnosis',color_discrete_sequence=['#A865C9','#f6abb6'])


In [201]:
px.histogram(data_frame=df,x=df.compactness_mean,color='diagnosis',color_discrete_sequence=['#A865C9','#f6abb6'])

#Data Preprocessing

In [202]:
df.drop('id', axis=1, inplace=True)

In [203]:
df['diagnosis'] = (df['diagnosis'] == 'M').astype(int)

In [204]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [205]:
df['diagnosis'].value_counts()

,count
diagnosis,
0,357
1,212


In [206]:
corr = df.corr()

In [207]:
corr

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
diagnosis,1.000000,0.730029,0.415185,0.742636,0.708984,0.358560,0.596534,0.696360,0.776614,0.330499,...,0.776454,0.456903,0.782914,0.733825,0.421465,0.590998,0.659610,0.793566,0.416294,0.323872
radius_mean,0.730029,1.000000,0.323782,0.997855,0.987357,0.170581,0.506124,0.676764,0.822529,0.147741,...,0.969539,0.297008,0.965137,0.941082,0.119616,0.413463,0.526911,0.744214,0.163953,0.007066
texture_mean,0.415185,0.323782,1.000000,0.329533,0.321086,-0.023389,0.236702,0.302418,0.293464,0.071401,...,0.352573,0.912045,0.358040,0.343546,0.077503,0.277830,0.301025,0.295316,0.105008,0.119205
perimeter_mean,0.742636,0.997855,0.329533,1.000000,0.986507,0.207278,0.556936,0.716136,0.850977,0.183027,...,0.969476,0.303038,0.970387,0.941550,0.150549,0.455774,0.563879,0.771241,0.189115,0.051019
area_mean,0.708984,0.987357,0.321086,0.986507,1.000000,0.177028,0.498502,0.685983,0.823269,0.151293,...,0.962746,0.287489,0.959120,0.959213,0.123523,0.390410,0.512606,0.722017,0.143570,0.003738
smoothness_mean,0.358560,0.170581,-0.023389,0.207278,0.177028,1.000000,0.659123,0.521984,0.553695,0.557775,...,0.213120,0.036072,0.238853,0.206718,0.805324,0.472468,0.434926,0.503053,0.394309,0.499316
compactness_mean,0.596534,0.506124,0.236702,0.556936,0.498502,0.659123,1.000000,0.883121,0.831135,0.602641,...,0.535315,0.248133,0.590210,0.509604,0.565541,0.865809,0.816275,0.815573,0.510223,0.687382
concavity_mean,0.696360,0.676764,0.302418,0.716136,0.685983,0.521984,0.883121,1.000000,0.921391,0.500667,...,0.688236,0.299879,0.729565,0.675987,0.448822,0.754968,0.884103,0.861323,0.409464,0.514930
concave points_mean,0.776614,0.822529,0.293464,0.850977,0.823269,0.553695,0.831135,0.921391,1.000000,0.462497,...,0.830318,0.292752,0.855923,0.809630,0.452753,0.667454,0.752399,0.910155,0.375744,0.368661
symmetry_mean,0.330499,0.147741,0.071401,0.183027,0.151293,0.557775,0.602641,0.500667,0.462497,1.000000,...,0.185728,0.090651,0.219169,0.177193,0.426675,0.473200,0.433721,0.430297,0.699826,0.438413


In [208]:
# Get the absolute value of the correlation
cor_target = abs(corr["diagnosis"])

In [209]:
# Select highly correlated features (thresold = 0.2)
relevant_features = cor_target[cor_target>0.2]

In [210]:
names = [index for index, value in relevant_features.items()]

In [211]:
names

['diagnosis',
 'radius_mean',
 'texture_mean',
 'perimeter_mean',
 'area_mean',
 'smoothness_mean',
 'compactness_mean',
 'concavity_mean',
 'concave points_mean',
 'symmetry_mean',
 'radius_se',
 'perimeter_se',
 'area_se',
 'compactness_se',
 'concavity_se',
 'concave points_se',
 'radius_worst',
 'texture_worst',
 'perimeter_worst',
 'area_worst',
 'smoothness_worst',
 'compactness_worst',
 'concavity_worst',
 'concave points_worst',
 'symmetry_worst',
 'fractal_dimension_worst']

In [212]:
names.remove('diagnosis')

In [213]:
X = df[names]
y = df['diagnosis']

In [214]:
y.value_counts()

,count
diagnosis,
0,357
1,212


In [215]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42) #split the  data into traing and validating


In [216]:
X_train.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,radius_se,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
68,9.029,17.33,58.79,250.5,0.10660,0.14130,0.31300,0.04375,0.2111,0.3274,...,10.31,22.65,65.50,324.7,0.14820,0.43650,1.25200,0.17500,0.4228,0.11750
181,21.090,26.57,142.70,1311.0,0.11410,0.28320,0.24870,0.14960,0.2395,0.6298,...,26.68,33.48,176.50,2089.0,0.14910,0.75840,0.67800,0.29030,0.4098,0.12840
63,9.173,13.86,59.20,260.9,0.07721,0.08751,0.05988,0.02180,0.2341,0.4098,...,10.01,19.23,65.59,310.1,0.09836,0.16780,0.13970,0.05087,0.3282,0.08490
248,10.650,25.22,68.01,347.0,0.09657,0.07234,0.02379,0.01615,0.1897,0.2497,...,12.25,35.19,77.98,455.7,0.14990,0.13980,0.11250,0.06136,0.3409,0.08147
60,10.170,14.88,64.55,311.9,0.11340,0.08061,0.01084,0.01290,0.2743,0.5158,...,11.02,17.45,69.86,368.6,0.12750,0.09866,0.02168,0.02579,0.3557,0.08020


In [217]:
scaler = StandardScaler() #create an instance of standard scaler

X_train_scaled = scaler.fit_transform(X_train)  # Kết hợp fit và transform cho dữ liệu huấn luyện
X_test_scaled = scaler.transform(X_test)

In [218]:
y_train.head()

,diagnosis
68,0
181,1
63,0
248,0
60,0


In [219]:
def train_evaluate_model(model, X_train, y_train, X_test,y_test):
    '''
    Keyword arguments:
    X -- Training data
    y -- Traing labels

    returns a dataframe for evaluating metrics
    '''

    model.fit(X_train, y_train)  #fit the model instance


    predictions = model.predict(X_test) # calculate predictions

    #compute metrics for evaluation
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    balanced_accuracy = balanced_accuracy_score(y_test, predictions)

    #create a dataframe to visualize the results
    eval_df = pd.DataFrame([[accuracy, f1, precision, recall, balanced_accuracy]], columns=['accuracy', 'f1_score', 'precision', 'recall', 'balanced_accuracy'])
    return eval_df

##Logistic Regression

In [220]:
lg = LogisticRegression()

results = train_evaluate_model(lg, X_train_scaled, y_train, X_test_scaled, y_test)

In [221]:
results.index = ['LogisticRegression']
results.sort_values(by='f1_score',ascending=False).style.background_gradient(cmap = sns.color_palette("ch:s=-.2,r=.6", as_cmap=True))

,accuracy,f1_score,precision,recall,balanced_accuracy
LogisticRegression,0.973684,0.964706,0.976190,0.953488,0.969702


##Decision Tree


In [222]:
decision_tree = DecisionTreeClassifier()
decision_tree_results = train_evaluate_model(decision_tree,X_train_scaled, y_train, X_test_scaled, y_test)
decision_tree_results.index = ['DecisionTree']

#KNN

In [223]:
KNN = KNeighborsClassifier(n_neighbors=12)

In [224]:
knn = train_evaluate_model(KNN, X_train_scaled, y_train, X_test_scaled, y_test)
knn.index =['KNearsNeighbors']

#RandomForestClassifer¶

In [225]:
rfc = RandomForestClassifier()

In [226]:
rfc_result = train_evaluate_model(rfc, X_train_scaled, y_train, X_test_scaled, y_test)
rfc_result.index = ['RandomForest']

#XGBoost

In [227]:
xgboost = xgb.XGBClassifier()
xgboost_result = train_evaluate_model(xgboost,X_train_scaled, y_train, X_test_scaled, y_test)
xgboost_result.index = ['XGBoost']

#Naive Bayes

In [228]:
Naive_Bayes = GaussianNB()
Naive_Bayes_result = train_evaluate_model(Naive_Bayes,X_train_scaled, y_train, X_test_scaled, y_test)
Naive_Bayes_result.index = ['NaiveBayes']

#NN

In [229]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

# Giả sử X có shape (num_samples, num_features)
input_shape = X_train.shape[1]  # Lấy số feature từ tập huấn luyện

# Khởi tạo bộ khởi tạo trọng số
initializer = tf.keras.initializers.RandomNormal(seed=0)

# Định nghĩa lịch trình học
def scheduler(epoch, lr):
    if epoch < 50:
        return float(lr)  # Chuyển đổi lr thành float
    else:
        return float(lr * tf.math.exp(-0.1).numpy())

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Định nghĩa mô hình mạng nơ-ron
inputs = Input(shape=(input_shape,))  # Chú ý phải truyền tuple (input_shape,)
x = Dense(16, activation='relu', kernel_initializer=initializer)(inputs)
x = Dropout(0.4)(x)
x = Dense(16, activation='relu', kernel_initializer=initializer)(x)
x = Dropout(0.4)(x)
x = Dense(25, activation='relu', kernel_initializer=initializer)(x)
outputs = Dense(1, activation='sigmoid', kernel_initializer=initializer)(x)

model = Model(inputs=inputs, outputs=outputs)

# Biên dịch mô hình
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình với callback
model.fit(X_train_scaled, y_train, epochs=140, callbacks=[callback])

# Dự đoán trên tập test
predictions = model.predict(X_test_scaled)


Epoch 1/140
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5438 - loss: 0.6925 - learning_rate: 0.0010
Epoch 2/140
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6451 - loss: 0.6887 - learning_rate: 0.0010
Epoch 3/140
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7987 - loss: 0.6761 - learning_rate: 0.0010
Epoch 4/140
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9027 - loss: 0.6367 - learning_rate: 0.0010
Epoch 5/140
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9244 - loss: 0.5322 - learning_rate: 0.0010
Epoch 6/140
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9236 - loss: 0.3854 - learning_rate: 0.0010
Epoch 7/140
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9335 - loss: 0.2411 - learning_rate: 0.0010
Epoch 8/140
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9513 - loss: 0.1711 - learning_rate: 0.0010
Epoch 9/140
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9531 - loss: 0.1415 - learning_rate: 0.0010
Epoch 10/140
15/15 

In [230]:
# Generate binary predictions for the test data using the trained neural network model
predictions = model.predict(X_test_scaled)
# convert predicted probabilities to binary predictions Note that a threshold of 0.4 was used as recall is more important for medical diagnosis
predictions = (predictions > 0.4)

# Calculate evaluation metrics using the binary predictions and true labels for the test data
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
balanced_accuracy = balanced_accuracy_score(y_test, predictions)

# Create a pandas dataframe containing the evaluation metric values as a single row
nn_df = pd.DataFrame([[accuracy, f1, precision, recall, balanced_accuracy]],
                     columns=['accuracy', 'f1_score', 'precision', 'recall', 'balanced_accuracy'])

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


In [231]:
nn_df.index = ['NeuralNetwork']

In [232]:
results = pd.concat([results, decision_tree_results,knn,rfc_result,xgboost_result,xgboost_result,Naive_Bayes_result,nn_df])
results

,accuracy,f1_score,precision,recall,balanced_accuracy
LogisticRegression,0.973684,0.964706,0.976190,0.953488,0.969702
DecisionTree,0.912281,0.886364,0.866667,0.906977,0.911235
KNearsNeighbors,0.956140,0.941176,0.952381,0.930233,0.951032
RandomForest,0.964912,0.952381,0.975610,0.930233,0.958074
XGBoost,0.956140,0.941176,0.952381,0.930233,0.951032
XGBoost,0.956140,0.941176,0.952381,0.930233,0.951032
NaiveBayes,0.964912,0.952381,0.975610,0.930233,0.958074
NeuralNetwork,0.991228,0.988235,1.000000,0.976744,0.988372
